In [1]:
import os
from get_input import get_input

os.chdir('/home/soup/Desktop/Projects/advent-of-code')

In [2]:

import re
from itertools import islice

In [3]:
input_str = get_input(2015, 12)

Found a datafile!


In [4]:
pattern = r'[{}\[\]\d,-]|\bred\b'

matches = re.findall(pattern, input_str)
output_str = ''.join(matches)

output_str

'{{{-39,119,{65,,,},,{6,{,8,[red,{37},,84,,,[24,45,,,56,],]},,{85},[109,66,[,,21,-30],,,-43,{,red,,-22,[71,red,30,,red,26,120],[red]},red]},{,,,,,{red,{115,137,},-25,,{161,[,56,129,-31,,red,,105,,130]},142,194,122,-16},173,[,,54,-9],-23},{110,,[{155},156,,94,],{91,-18,red,[,,,190,,158,,,,4],143,,145,[,37,,-22,{30,78,196,84},-7,[]]}}},[[{0,108,,{59,119,104,167,}},[189,,121,[[,,,-17,192,red],{},,{42,,red},{78,,,-9,,115,,,red}],57,],],[[,[1,53,],{[,-12,,,,136,173],,-43,{144,133,,154,,127,194},{52,-43,,-45,,150,-12,91,6},{,,,112,174,,90},177,},red,,96],,,[{[red,red,46,red],[,193,54,]},[,8,1,[,84,],155,,151,,196],,red,{[,,,61,,39,-2,46,red,54]},,128]]]},{[{[],93,,{{,},-4},},171,103,[13,,[[51,,,{85,103,},97,{,-11,62,,,47,83,119,180,136},{177},80],{{,,3,197},130,red,red,-44,-15,64,125,82,}],{[,42,[,197,,,,11,],189,,,,],{149,69},128,[[,150,45,86,red,-8,41,,]],[[-4,127,,,124,112,196,,161,40],37,0,,-30,-43,[-24,,142,,red],,66],{,[,91,182,20,,159,46,55,141],{173,-40,,},[67,80,27,-15]},red,{,99,}}

In [5]:
curly_stack = []
square_stack = []
pairs = {}

for index, char in enumerate(input_str):
    if char == '{':
        curly_stack.append(index)
    elif char == '[':
        square_stack.append(index)
    elif char == '}':
        if len(curly_stack) == 0:
            print(f"unclosed curly bracket at index: {index}")
        else:
            pairs[curly_stack.pop()] = (index, 'curly')
    elif char == ']':
        if len(square_stack) == 0:
            print(f"unclosed square bracket at index: {index}")
        else:
            pairs[square_stack.pop()] = (index, 'square')

In [6]:
l = list(input_str)

for key, (end, bracket) in pairs.items():
    substring = input_str[key:end+1]
    if 'red' in substring:
        if bracket == 'square':
            new_substring = substring.replace('red', '$$$')
            l[key:end+1] = list(new_substring)
            input_str = "".join(l)
        elif bracket == 'curly':
            new_substring = "$" * (len(substring) - 2)
            l[key +1:end] = list(new_substring)
            input_str = "".join(l)

fixed_string = "".join(l)


In [7]:
fixed_string

'{"e":{"a":{"e":-39,"c":119,"a":{"c":65,"a":"orange","b":"green","d":"orange"},"g":"violet","b":{"e":6,"c":{"c":"violet","a":8,"b":["$$$",{"a":37},"green",84,"yellow","green",[24,45,"blue","blue",56,"yellow"],"orange"]},"a":"violet","b":{"a":85},"d":[109,66,["yellow","violet",21,-30],"violet","blue",-43,{$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$},"$$$"]},"d":{"e":"violet","a":"blue","d":"blue","c":"blue","h":"orange","b":{$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$},"g":173,"f":["orange","green",54,-9],"i":-23},"f":{"c":110,"a":"yellow","b":[{"a":155},156,"violet",94,"yellow"],"d":{$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$}}},"b":[[{"c":0,"a":108,

In [8]:
def str_sum(s:str) -> int:
    pattern = r'-?\d+'
    nums = re.findall(pattern, s)
    
    return sum(int(x) for x in nums)

In [9]:
print(f"The sum of all the integers in the input is: {str_sum(fixed_string)}")

The sum of all the integers in the input is: 68466
